In [1]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM

Using TensorFlow backend.


In [2]:
import numpy as np
import pandas as pd

In [3]:
import pickle

Embeding

In [4]:
max_features = 20000
maxlen = 20
embedding_size = 128

LSTM

In [5]:
lstm_output_size = 128

Training

In [6]:
batch_size = 32
epochs = 20

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

In [8]:
ds = pickle.load(open('./tweet_seq.txt','rb'))

In [9]:
ds.head()

,CATEGORY,TWEET
0,1,"[268, 456, 146, 2, 469, 4487]"
1,1,"[3, 3, 3, 3, 2263, 8, 9821, 3, 3047]"
2,0,"[3, 80, 173, 35, 1, 14, 39, 2264, 8, 9822, 226..."
3,1,"[3, 108, 41, 21, 470, 10, 660, 6, 216, 300, 6,..."
4,1,"[3, 3, 559, 107, 29, 579, 29, 418]"


In [10]:
ds.count()

CATEGORY    15306
TWEET       15306
dtype: int64

In [11]:
print('Pad sequences (samples x time)')
tweet_sec = sequence.pad_sequences(ds['TWEET'], maxlen=maxlen)
print('tweet shape:', tweet_sec.shape)

Pad sequences (samples x time)
tweet shape: (15306, 20)


In [12]:
def build_arch():
    print('Build model...')
    model = Sequential()
    model.add(Embedding(max_features, embedding_size))
    model.add(LSTM(lstm_output_size,
                   activation='tanh', 
                   recurrent_activation='hard_sigmoid', 
                   dropout=0.2, 
                   recurrent_dropout=0.2))
    model.add(Dense(1, activation='sigmoid'))
    return model

In [13]:
def cross_val(tweet_sec, target, nFolds):
    kFold = StratifiedKFold(n_splits=nFolds, shuffle=True)
    scores = []
    for train, test in kFold.split(tweet_sec,target):
        model = build_arch()
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
        model.fit(tweet_sec[train],target[train],
                  batch_size=batch_size,
                  epochs=epochs,
                  validation_split=0.1,
                  verbose=1)
        score, acc = model.evaluate(tweet_sec[test], target[test],batch_size=batch_size)
        scores.append(acc * 100)
    return scores   

In [14]:
#Training with the 100% of the data set
def train_model(tweet_sec,target):
    model = build_arch()
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model.fit(tweet_sec, target,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1)

In [15]:
results = cross_val(tweet_sec,ds['CATEGORY'],10)

Build model...
Train on 12396 samples, validate on 1378 samples
Epoch 1/20
12396/12396 [==============================] - 25s 2ms/step - loss: 0.4021 - acc: 0.8187 - val_loss: 0.2782 - val_acc: 0.8853
Epoch 2/20
12396/12396 [==============================] - 23s 2ms/step - loss: 0.1787 - acc: 0.9342 - val_loss: 0.2943 - val_acc: 0.8919
Epoch 3/20
12396/12396 [==============================] - 23s 2ms/step - loss: 0.0962 - acc: 0.9657 - val_loss: 0.3871 - val_acc: 0.8694
Epoch 4/20
12396/12396 [==============================] - 24s 2ms/step - loss: 0.0535 - acc: 0.9820 - val_loss: 0.5289 - val_acc: 0.8628
Epoch 5/20
12396/12396 [==============================] - 23s 2ms/step - loss: 0.0376 - acc: 0.9864 - val_loss: 0.5496 - val_acc: 0.8563
Epoch 6/20
12396/12396 [==============================] - 24s 2ms/step - loss: 0.0258 - acc: 0.9911 - val_loss: 0.6448 - val_acc: 0.8527
Epoch 7/20
12396/12396 [==============================] - 24s 2ms/step - loss: 0.0252 - acc: 0.9914 - val_loss: 0.

12396/12396 [==============================] - 23s 2ms/step - loss: 0.0051 - acc: 0.9981 - val_loss: 1.1229 - val_acc: 0.8389
Epoch 19/20
12396/12396 [==============================] - 23s 2ms/step - loss: 0.0062 - acc: 0.9981 - val_loss: 0.9222 - val_acc: 0.8186
Epoch 20/20
1532/1532 [==============================] - 0s 253us/step
Build model...
Train on 12398 samples, validate on 1378 samples
Epoch 1/20
12398/12398 [==============================] - 24s 2ms/step - loss: 0.3966 - acc: 0.8209 - val_loss: 0.2937 - val_acc: 0.8832
Epoch 2/20
12398/12398 [==============================] - 23s 2ms/step - loss: 0.1788 - acc: 0.9356 - val_loss: 0.3133 - val_acc: 0.8795
Epoch 3/20
12398/12398 [==============================] - 23s 2ms/step - loss: 0.0935 - acc: 0.9668 - val_loss: 0.3865 - val_acc: 0.8694
Epoch 4/20
12398/12398 [==============================] - 23s 2ms/step - loss: 0.0558 - acc: 0.9809 - val_loss: 0.5224 - val_acc: 0.8440
Epoch 5/20
12398/12398 [=============================

Epoch 15/20
12398/12398 [==============================] - 23s 2ms/step - loss: 0.0061 - acc: 0.9981 - val_loss: 0.9954 - val_acc: 0.8360
Epoch 16/20
12398/12398 [==============================] - 24s 2ms/step - loss: 0.0035 - acc: 0.9990 - val_loss: 1.0212 - val_acc: 0.8360
Epoch 17/20
12398/12398 [==============================] - 24s 2ms/step - loss: 0.0022 - acc: 0.9995 - val_loss: 1.1270 - val_acc: 0.8345
Epoch 18/20
12398/12398 [==============================] - 23s 2ms/step - loss: 0.0059 - acc: 0.9984 - val_loss: 0.8796 - val_acc: 0.8331
Epoch 19/20
12398/12398 [==============================] - 23s 2ms/step - loss: 0.0112 - acc: 0.9963 - val_loss: 0.9150 - val_acc: 0.8338
Epoch 20/20
1530/1530 [==============================] - 0s 256us/step
Build model...
Train on 12398 samples, validate on 1378 samples
Epoch 1/20
12398/12398 [==============================] - 25s 2ms/step - loss: 0.3969 - acc: 0.8217 - val_loss: 0.3255 - val_acc: 0.8643
Epoch 2/20
12398/12398 [==============

Epoch 12/20
12398/12398 [==============================] - 23s 2ms/step - loss: 0.0116 - acc: 0.9964 - val_loss: 0.8891 - val_acc: 0.8374
Epoch 13/20
12398/12398 [==============================] - 23s 2ms/step - loss: 0.0092 - acc: 0.9973 - val_loss: 1.0163 - val_acc: 0.8382
Epoch 14/20
12398/12398 [==============================] - 23s 2ms/step - loss: 0.0081 - acc: 0.9973 - val_loss: 1.1561 - val_acc: 0.8287
Epoch 15/20
12398/12398 [==============================] - 23s 2ms/step - loss: 0.0089 - acc: 0.9979 - val_loss: 0.9903 - val_acc: 0.8309
Epoch 16/20
12398/12398 [==============================] - 23s 2ms/step - loss: 0.0069 - acc: 0.9974 - val_loss: 1.0431 - val_acc: 0.8353
Epoch 17/20
12398/12398 [==============================] - 23s 2ms/step - loss: 0.0076 - acc: 0.9975 - val_loss: 1.0762 - val_acc: 0.8309
Epoch 18/20
12398/12398 [==============================] - 23s 2ms/step - loss: 0.0038 - acc: 0.9986 - val_loss: 1.0563 - val_acc: 0.8425
Epoch 19/20
12398/12398 [=========

In [16]:
print("Acc: %.2f (+/- %.2f)" %(np.mean(results), np.std(results)))

Acc: 85.15 (+/- 1.07)


In [17]:
for i in range (0,len(results)):
    print "Model %d, acc: %.2f " %(i+1,results[i])

SyntaxError: invalid syntax (<ipython-input-17-42654225b0b8>, line 2)

In [22]:
trained_model = train_model(tweet_sec,ds['CATEGORY'])

Build model...
Epoch 1/20
15306/15306 [==============================] - 34s - loss: 0.3845 - acc: 0.8339    
Epoch 2/20
15306/15306 [==============================] - 33s - loss: 0.1770 - acc: 0.9347    
Epoch 3/20
15306/15306 [==============================] - 32s - loss: 0.0932 - acc: 0.9654    
Epoch 4/20
15306/15306 [==============================] - 32s - loss: 0.0588 - acc: 0.9795    
Epoch 5/20
15306/15306 [==============================] - 32s - loss: 0.0406 - acc: 0.9856    
Epoch 6/20
15306/15306 [==============================] - 32s - loss: 0.0369 - acc: 0.9873    
Epoch 7/20
15306/15306 [==============================] - 32s - loss: 0.0230 - acc: 0.9918    
Epoch 8/20
15306/15306 [==============================] - 31s - loss: 0.0195 - acc: 0.9929    
Epoch 9/20
15306/15306 [==============================] - 32s - loss: 0.0213 - acc: 0.9922    
Epoch 10/20
15306/15306 [==============================] - 32s - loss: 0.0128 - acc: 0.9955    
Epoch 11/20
15306/15306 [=========

In [23]:
model_json = trained_model.model.to_json()
with open("./LSTM.json",'w') as json_file:
    json_file.write(model_json)
trained_model.model.save_weights("./LSTM_weights.h5")